# Gold Layer ETL
Time to ingest data into empty schemas once again. This time we are bringing the data from the silver layer into the gold layer. 

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

### a) Customer Profile Table
Which features of the customer base are the most relevant is maximizing expenditure?

In [0]:
%sql
INSERT INTO gold.customer_profile
SELECT
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain,

    -- Total Spending = MntWines + MntFruits + ... + MntSweetProds (Obs: *no* MntGoldProds, as its a different kind of category)  
    SUM(
        f.MntWines +
        f.MntFruits +
        f.MntMeatProducts +
        f.MntFishProducts +
        f.MntSweetProducts
    ) AS total_spending,

    -- Average Spending 
    AVG(
        f.MntWines +
        f.MntFruits +
        f.MntMeatProducts +
        f.MntFishProducts +
        f.MntSweetProducts
    ) AS avg_spent_per_customer

FROM silver.FACT_Sales f
JOIN silver.DIM_Customer c
  ON f.fk_customer = c.id_customer

GROUP BY
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain;


In [0]:
%sql
-- Test
SELECT * FROM gold.customer_profile 
ORDER BY total_spending DESC 

In [0]:
%sql
SELECT * FROM gold.customer_profile 
ORDER BY total_spending DESC LIMIT 10

Not very useful, as the combinations are too many, but we can already see which (highly specific) groups are the biggest spenders. The master's graduates who are single and have no offspring take the lead.

In [0]:
df_gold = spark.table("gold.customer_profile")

df_gold.select(
    F.corr("income", "total_spending").alias("corr_income_spending")
).display()


As expected, the correlation between Income and Total Spending is positive and moderate to strong, at 0.64

In [0]:
%sql
SELECT
  education,
  ROUND(AVG(total_spending), 2) AS avg_spending
FROM gold.customer_profile
GROUP BY education
ORDER BY avg_spending DESC;

Also to be expected, we find PhDs at the top of the spending ranking and people with Basic education at the bottom. Usually these are correlated with Income.

In [0]:
%sql
SELECT
  marital_status,
  ROUND(AVG(total_spending),2) AS avg_spending
FROM gold.customer_profile
GROUP BY marital_status
ORDER BY avg_spending DESC;


It is notable that the 'Married' marital status is the very last one in spending. Also the incorrect categories are gone, but the new 'Unknown' category is not here for some reason...

In [0]:
df_gold = spark.table("gold.customer_profile")

df_gold.select(
    F.corr("kidhome", "total_spending").alias("corr_income_spending")
).display()


In [0]:
df_gold = spark.table("gold.customer_profile")

df_gold.select(
    F.corr("teenhome", "total_spending").alias("corr_income_spending")
).display()


Interestingly, having offspring is negatively correlated with Total Spending. Additionaly, the lowering strength of the correlation might suggest a relation to the maturity of the child.

In [0]:
%sql
SELECT
  kidhome,
  ROUND(AVG(total_spending),2) AS avg_spending
FROM gold.customer_profile
GROUP BY kidhome
ORDER BY kidhome;


In [0]:
%sql
SELECT
  teenhome,
  ROUND(AVG(total_spending),2) AS avg_spending
FROM gold.customer_profile
GROUP BY teenhome
ORDER BY teenhome;


In summary:

In [0]:
pdf = df_gold.select(
    "total_spending",
    "income",
    "kidhome",
    "teenhome"
).toPandas()

sns.heatmap(pdf.corr(), annot=True, cmap="coolwarm")
plt.show()

Evidently total spending is positively correlated with income, but it is intriguing that is is *negatively* correlated with having kids at home. Having a teen at home seems not to be relevant. This could suggest that it is wiser to market the company's products to young adults, with income but no family yet.

### b) Promo Effectiveness Table
Which features of the customer base are most relevant to the effectiveness of promotional campaigns?

In [0]:
%skip
%sql
CREATE OR REPLACE TABLE gold.promo_acceptance_rate
USING DELTA
AS

SELECT
    campaign,
    COUNT(*) AS total_customers,
    SUM(accepted) AS accepted_customers,
    SUM(accepted) / COUNT(*) AS acceptance_rate
FROM (
    SELECT 'Campaign_1' AS campaign, AcceptedCmp1 AS accepted FROM silver.raw_data
    UNION ALL
    SELECT 'Campaign_2', AcceptedCmp2 FROM silver.raw_data
    UNION ALL
    SELECT 'Campaign_3', AcceptedCmp3 FROM silver.raw_data
    UNION ALL
    SELECT 'Campaign_4', AcceptedCmp4 FROM silver.raw_data
    UNION ALL
    SELECT 'Campaign_5', AcceptedCmp5 FROM silver.raw_data
    UNION ALL
    SELECT 'Last_Campaign', Response FROM silver.raw_data
) t
GROUP BY campaign;


In [0]:
%skip
%sql
INSERT INTO gold.promo_acceptance_rate
SELECT
    'Campaign 1' AS campaign_num,
    AVG(p.AcceptedCmp1) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.promo_acceptance_rate
SELECT
    'Campaign 2' AS campaign_num,
    AVG(p.AcceptedCmp2) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.promo_acceptance_rate
SELECT
    'Campaign 3' AS campaign_num,
    AVG(p.AcceptedCmp3) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.promo_acceptance_rate
SELECT
    'Campaign 4' AS campaign_num,
    AVG(p.AcceptedCmp4) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.promo_acceptance_rate
SELECT
    'Campaign 5' AS campaign_num,
    AVG(p.AcceptedCmp5) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.promo_acceptance_rate
SELECT
    'Last Campaign (6)' AS campaign_num,
    AVG(p.Response) AS acceptance_rate,
    COUNT(*) AS total_customers
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;




In [0]:
%skip
display(spark.table("gold.promo_acceptance_rate"))

In [0]:
%skip
%sql
SELECT * 
FROM gold.promo_acceptance_rate
ORDER BY acceptance_rate DESC

In [0]:
%sql
INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_1' AS campaign_num,
    COUNT(DISTINCT f.fk_customer) AS total_customers,
    SUM(CASE WHEN p.AcceptedCmp1 = 1 THEN 1 ELSE 0 END) AS accepted_customers,
    SUM(CASE WHEN p.AcceptedCmp1 = 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer) AS acceptance_rate
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_2',
    COUNT(DISTINCT f.fk_customer),
    SUM(CASE WHEN p.AcceptedCmp2 = 1 THEN 1 ELSE 0 END),
    SUM(CASE WHEN p.AcceptedCmp2 = 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer)
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_3',
    COUNT(DISTINCT f.fk_customer),
    SUM(CASE WHEN p.AcceptedCmp3 = 1 THEN 1 ELSE 0 END),
    SUM(CASE WHEN p.AcceptedCmp3 = 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer)
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_4',
    COUNT(DISTINCT f.fk_customer),
    SUM(CASE WHEN p.AcceptedCmp4 = 1 THEN 1 ELSE 0 END),
    SUM(CASE WHEN p.AcceptedCmp4 = 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer)
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_5',
    COUNT(DISTINCT f.fk_customer),
    SUM(CASE WHEN p.AcceptedCmp5 = 1 THEN 1 ELSE 0 END),
    SUM(CASE WHEN p.AcceptedCmp5 = 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer)
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;

INSERT INTO gold.campaign_acceptance
SELECT
    'Campaign_6',
    COUNT(DISTINCT f.fk_customer),
    SUM(CASE WHEN p.Response = 1 THEN 1 ELSE 0 END),
    SUM(CASE WHEN p.Response= 1 THEN 1 ELSE 0 END)
      / COUNT(DISTINCT f.fk_customer)
FROM silver.FACT_Sales f
JOIN silver.DIM_Promo p
  ON f.fk_promo = p.id_promo;


In [0]:
display(spark.table("gold.campaign_acceptance"))

In [0]:
%sql
SELECT * 
FROM gold.campaign_acceptance
ORDER BY acceptance_rate DESC

Fortunately for the company, its marketing efforts appear to be improving, as the most recent campaign achieved the highest acceptance rate.

### c) Product Performance Table
Which are the best performing products?

In [0]:
%sql
INSERT OVERWRITE gold.product_performance
WITH product_sales AS (

    -- Wines
    SELECT
        'Wines' AS product_category,
        fk_customer,
        MntWines AS revenue
    FROM silver.FACT_Sales
    WHERE MntWines > 0

    UNION ALL

    -- Fruits
    SELECT
        'Fruits',
        fk_customer,
        MntFruits
    FROM silver.FACT_Sales
    WHERE MntFruits > 0

    UNION ALL

    -- Meat
    SELECT
        'Meat Products',
        fk_customer,
        MntMeatProducts
    FROM silver.FACT_Sales
    WHERE MntMeatProducts > 0

    UNION ALL

    -- Fish
    SELECT
        'Fish Products',
        fk_customer,
        MntFishProducts
    FROM silver.FACT_Sales
    WHERE MntFishProducts > 0

    UNION ALL

    -- Sweets
    SELECT
        'Sweet Products',
        fk_customer,
        MntSweetProducts
    FROM silver.FACT_Sales
    WHERE MntSweetProducts > 0
),

aggregated AS (
    SELECT
        product_category,
        SUM(revenue)                    AS total_revenue,
        AVG(revenue)                    AS avg_revenue_per_customer,
        COUNT(DISTINCT fk_customer)     AS purchase_count
    FROM product_sales
    GROUP BY product_category
)

SELECT
    product_category,
    total_revenue,
    avg_revenue_per_customer,
    purchase_count,
    DENSE_RANK() OVER (ORDER BY total_revenue DESC) AS revenue_rank
FROM aggregated;


In [0]:
display(spark.table("gold.product_performance"))

Not much can be concluded from just this table without greater context. Perhaps the company should stop selling fruits; perhaps it needs to bolster its marketing for fruits. Regardless, having awareness of the asymmetries between the different product categories would be absolutely essential for strategic thinking.

### d) Purchase Channel Summary Table
Which is the most popular method of purchasing?


In [0]:
%sql
CREATE OR REPLACE TABLE gold.purchase_channel_summary AS
SELECT
    ROUND(AVG(NumWebPurchases), 2) AS avg_web,
    ROUND(AVG(NumStorePurchases), 2) AS avg_store,
    ROUND(AVG(NumCatalogPurchases), 2) AS avg_catalog
FROM silver.fact_sales;

In [0]:
display(spark.table("gold.purchase_channel_summary"))

The average consumer purchases the least from the company's catalog and the most from physical stores. While for now there can only be conjecture about the reasons why, it is nevertheless a valuable insight taht might, for instance, suggest that catalogs are not worth mantaining, or that stores should at the very least receive the same level of attention and not be neglected.